In [32]:
import json
import os
from collections import defaultdict
from mp3_sorter_tagger.helpers import check_name_change_safe, split_album_year_name, split_album_name_type

def get_library():
    root = '/mnt/d/Soulseek'
    folders = {}
    for band in os.listdir(root):
        if band == '_VA':
            continue
        folders[band] = {}
        for album in os.listdir(os.path.join(root, band)):
            path = os.path.join(root, band, album)
            if os.path.isdir(path):
                folders[band][album] = os.listdir(path)
    return folders

folders = get_library()

with open('folders.json', 'w') as json_file:
    json.dump(folders, json_file)


with open('folders.json', 'r') as json_file:
    folders = json.load(json_file)

path = '/home/naztom/tvd/mp3_sorter_tagger/data/'
os.chdir(path)

# to_rename = defaultdict(dict)
# all_good = defaultdict(dict)
# incorrect_title = defaultdict(dict)

# for band in os.listdir():
#     os.chdir(path + band)
#     for folder in os.listdir():
#         year, name = split_album_year_name(folder)
#         name, type, name_original, name_changed = split_album_name_type(name)
#         full = ' - '.join([str(year), name]) if year else name
#         full += f' [{type}]' if type else ''
#         if name_changed and not check_name_change_safe(name, name_original):
#             incorrect_title[band][folder] = full
#         elif folder != full:
#             to_rename[band][folder] = full
#         else:
#             all_good[band][folder] = full

# display(incorrect_title)
# display(to_rename)
# display(all_good)



In [52]:
import re
pattern = r"^(?=.*\b(\d{4})\b)(?!.*\b\d{4}\b.*\b\d{4}\b).*"

for artist, albums in folders.items():
    nodate = []
    for album in albums:
        if not re.findall(pattern, album):
            nodate.append(album)
    if nodate:
        print(artist)
        for album in nodate:
            print('\t', album)

Angelspit
	 The Bastard Gods
Angeltheory
	 Black and Blue EP
	 Fatal Condition
	 Into the Fall
	 Re-Possession
	 Transmission __ ep
Apoptygma Berzerk
	 _live
Asche
	 Distorted Disco
Ashbury Heights
	 Morningstar in a Black Car
	 Take Cair Paramour
	 The Looking Glass Society
	 The Victorian Wallflowers
	 Three Cheers for the Newlydeads
Aslan Faction
	 Blunt Force Trauma
Bauhaus
	 1979-1983 Volume One
	 1979-1983 Volume Two
Beborn Beton
	 Darkness Falls Again
Black Tape for a Blue Girl
	 As One Aflame Laid Bare by Desire
	 Mesmerized by the Sirens
	 Remnants of a Deeper Purity
	 The Scarecrow
Blutengel
	 Demon Kiss
Borghesia
	 1988 - Ogolelo Mesto (RTV Ljubljana LD 1627)
Cenobita
	 Metamorfosis
Cesium_137
	 Luminous
Clan of Xymox
	 1994 Subsequent Pleasures (reissue 2001)
Client
	 City
	 Here and Now
	 Rock and Roll Machine
Decoded Feedback
	 1996 overdosing (remastered 2002)
Deine Lakaien
	 2. Singles & EPs
	 2002 - Album 2002 (White Lies Promo)
	 2003  1987 the early works
	 2003 - 19